# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Model Training</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/bitcoin/4_model_training.ipynb)

## 🗒️ This notebook is divided into the following sections:
1. Loading the training data
2. Train the model
3. Register model in Hopsworks model registry
4. Online model deployment
5. Use the deployment to make a prediciton

![part3](../../images/03_model.png)

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
!pip install -U hopsworks --quiet

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

---
### <span style="color:#ff5f27;"> 📝 Imports</span>

In [ ]:
from __future__ import print_function

import tensorflow as tf
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

%config InlineBackend.figure_format='retina'
%matplotlib inline

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina', quality=100)

import warnings
warnings.filterwarnings('ignore')

---

## <span style="color:#ff5f27;">🪝 Feature View and Training Dataset Retrieval</span>

In [ ]:
feature_view = fs.get_feature_view(
    name = 'bitcoin_feature_view',
    version = 1
)

In [ ]:
X_train, _ = feature_view.get_training_data(1)
X_val, _ = feature_view.get_training_data(2)
X_test, _ = feature_view.get_training_data(3)

In [ ]:
X_train.head(3)

In [ ]:
# lets remove redundant column "unix"
X_train.drop(columns=["unix"], inplace=True)
X_val.drop(columns=["unix"], inplace=True)
X_test.drop(columns=["unix"], inplace=True)

In [ ]:
y_train = X_train[["close"]]
y_val = X_val[["close"]]
y_test = X_test[["close"]]

---

## <span style="color:#ff5f27;">🤖 Time series model</span>

In [ ]:
# Now lets define Tensorflow Dataset as we are going to train keras tensorflow model

def windowed_dataset(dataset, target, window_size, batch_size):
    ds = dataset.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda x: x.batch(window_size))
    ds = ds.map(lambda window: tf.reshape(window[-1:], [-1, 33]))
        
    target_ds = target.window(window_size, shift=1, drop_remainder=True)
    target_ds = target_ds.flat_map(lambda window: window.batch(window_size))
    target_ds = target_ds.map(lambda window: window[-1:])
    
    ds = tf.data.Dataset.zip((ds, target_ds))
    ds = ds.batch(batch_size,True)
    ds = ds.prefetch(1)
    return ds

In [ ]:
training_dataset = tf.data.Dataset.from_tensor_slices(tf.cast(X_train.values, tf.float32)) 
training_target = tf.data.Dataset.from_tensor_slices(y_train.values.flatten().tolist()) 
training_dataset = training_dataset.repeat(500)
training_dataset = windowed_dataset(training_dataset, training_target, window_size=2, batch_size=16)
training_dataset

In [ ]:
validation_dataset = tf.data.Dataset.from_tensor_slices(tf.cast(X_val.values, tf.float32))
validation_target = tf.data.Dataset.from_tensor_slices(y_val.values.flatten().tolist()) 
training_dataset = training_dataset.repeat(500)
validation_dataset = windowed_dataset(validation_dataset, validation_target, window_size=2, batch_size=16)
validation_dataset

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices(tf.cast(X_test.values, tf.float32))
test_target = tf.data.Dataset.from_tensor_slices(y_test.values.flatten().tolist()) 
test_dataset = windowed_dataset(test_dataset, test_target, window_size=2, batch_size=1)
test_dataset

In [ ]:
def build_model(input_dim):
    inputs = tf.keras.layers.Input(shape=(input_dim[0],input_dim[1]))
    x = tf.keras.layers.Conv1D(filters = 128, kernel_size=1, padding='same', kernel_initializer="uniform")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)    
    x = tf.keras.layers.MaxPooling1D(pool_size=2, padding='same')(x)
    x = tf.keras.layers.Conv1D(filters = input_dim[1], kernel_size= 1,padding='same',  kernel_initializer="uniform")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)    
    x = tf.keras.layers.MaxPooling1D(pool_size=2, padding='same')(x)    

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(33, activation="relu", kernel_initializer="uniform")(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(1, activation="relu", kernel_initializer="uniform")(x)
    
    model = tf.keras.Model(inputs, x)
    model.summary()
    model.compile(loss='mse',optimizer='adam',metrics=['mae'])
    return model

In [ ]:
model = build_model([1, X_train.shape[1]])

In [ ]:
from timeit import default_timer as timer
start = timer()
history = model.fit(training_dataset,
                    epochs=10,
                    verbose=0,
                    steps_per_epoch=500,
                    validation_data=validation_dataset,
                    validation_steps=1,                    
                   )
end = timer()
print(end - start)

In [ ]:
history_dict = history.history
history_dict.keys()

### <span style='color:#ff5f27'>👮🏻‍♂️ Model Validation</span>

In [ ]:
loss_values = history_dict['mae']
val_loss_values = history_dict['val_mae']

loss_values50 = loss_values
val_loss_values50 = val_loss_values
epochs = range(1, len(loss_values50) + 1)
plt.plot(epochs, loss_values50, 'b',color = 'blue', label='Training loss')
plt.plot(epochs, val_loss_values50, 'b',color='red', label='Validation loss')
plt.rc('font', size = 18)
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.xticks(epochs)
fig = plt.gcf()
fig.set_size_inches(15,7)
plt.show()

In [ ]:
y_pred = model.predict(X_test.values.reshape(-1, 1, X_test.shape[1]))
y_pred[:5]

In [ ]:
plt.plot(y_pred,color='red', label='Predicted tested values')
plt.plot(y_test.values, color='blue', label='Actual values')
plt.xlabel('No. of Trading Days')
plt.ylabel('Close Value (scaled)')
plt.legend(loc='upper left')
fig = plt.gcf()
fig.set_size_inches(15, 5)
plt.show()

In [ ]:
from sklearn import metrics

print("R2:", metrics.r2_score(y_test, y_pred))

In [ ]:
colors = ['darkslategrey']

fig=plt.figure()
ax=fig.add_axes([0,0,1,1])

ax.plot(y_test, 'black')
ax.plot(y_pred, 'cyan')
ax.set_ylabel('$price$')
ax.set_xlabel('$time$')
ax.grid(True)
ax.legend(["actual", "pred"])

fig.tight_layout()

plt.grid(True)
plt.show()

In [ ]:
import inspect 
# Recall that you applied transformation functions, such as min max scaler and laber encoder. 
# Now you want to transform them back to human readable format.
feature_view.init_serving(1)
td_transformation_functions = feature_view._single_vector_server._transformation_functions

pred_y = pd.DataFrame(y_pred, columns=["close"])

for feature_name in td_transformation_functions:
    if feature_name == "close":
        td_transformation_function = td_transformation_functions[feature_name]
        sig, foobar_locals = inspect.signature(td_transformation_function.transformation_fn), locals()
        param_dict = dict([(param.name, param.default) for param in sig.parameters.values() if param.default != inspect._empty])
        if td_transformation_function.name == "min_max_scaler":
            pred_y[feature_name] = pred_y[feature_name].map(lambda x: x*(param_dict["max_value"]-param_dict["min_value"])+param_dict["min_value"])
            y_test[feature_name] = y_test[feature_name].map(lambda x: x*(param_dict["max_value"]-param_dict["min_value"])+param_dict["min_value"])

In [ ]:
colors = ['darkslategrey']

fig=plt.figure()
ax=fig.add_axes([0,0,1,1])

ax.plot(y_test, 'black')
ax.plot(pred_y, 'orange')
ax.set_ylabel('$price$')
ax.set_xlabel('$time$')
ax.grid(True)
ax.legend(["actual", "pred"])

fig.tight_layout()

plt.grid(True)
plt.show()

---
## <span style='color:#ff5f27'>🗄 Model Registry</span>

In [ ]:
export_path = "bitcoin_price_model"
print('Exporting trained model to: {}'.format(export_path))
    
tf.saved_model.save(model, export_path)

In [ ]:
mr = project.get_model_registry()
metrics={'loss': history_dict['val_mae'][0]} 

mr_model = mr.tensorflow.create_model(
    name="bitcoin_price_model",
    metrics=metrics,
    description="bitcoin daily price detection model.",
    input_example=[1613512800000]
)

In [ ]:
mr_model.save(export_path)

---
## <span style="color:#ff5f27;">🚀 Model Deployment</span>

In [ ]:
%%writefile btc_model_transformer.py

import os
import hsfs
import numpy as np

class Transformer(object):
    
    def __init__(self):        
        # get feature store handle
        fs_conn = hsfs.connection()
        self.fs = fs_conn.get_feature_store()
        
        # get feature views
        self.fv = self.fs.get_feature_view("bitcoin_feature_view", 1)
        
        # initialise serving
        self.fv.init_serving(1)

    def flat2gen(self, alist):
        for item in alist:
            if isinstance(item, list):
                for subitem in item: yield subitem
            else:
                yield item
        
    def preprocess(self, inputs):
        feature_vector = self.fv.get_feature_vector({"unix": inputs["inputs"]})
        feature_vector = [*feature_vector[:9], *feature_vector[10:]]
        return { "inputs" :  np.array(list(self.flat2gen(feature_vector))).reshape(-1, 1, len(feature_vector)).tolist() }

    def postprocess(self, outputs):
        return outputs    

In [ ]:
import os
from hsml.transformer import Transformer
dataset_api = project.get_dataset_api()

uploaded_file_path = dataset_api.upload("btc_model_transformer.py", "Models", overwrite=True)
transformer_script_path = os.path.join("/Projects", project.name, uploaded_file_path)
transformer_script = Transformer(script_file=transformer_script_path)

In [ ]:
# Use the model name from the previous notebook.
model = mr.get_model("bitcoin_price_model", version = 1)

In [ ]:
deployment = model.deploy(
    name="btcmodeldeployment",
    model_server="TENSORFLOW_SERVING", 
    serving_tool="KSERVE",
    transformer=transformer_script
)

The deployment has now been registered. Lets retrieve it from Hopsworks for demonstration purpose.

In [ ]:
ms = project.get_model_serving()

# get deployment object
deployment = ms.get_deployment("btcmodeldeployment")

In [ ]:
print("Deployment: " + deployment.name)
deployment.describe()

To start it you need to run:

In [ ]:
deployment.start(await_running=120)

For trouble shooting one can use get_logs method

In [ ]:
deployment.get_logs()

---
## <span style="color:#ff5f27;">🔮 Predicting</span>

Using the deployment let's use the input example that we registered together with the model to query the deployment.


In [ ]:
data = {
    "inputs": model.input_example[0]
}
data

In [ ]:
deployment.predict(data)

In [ ]:
# For trouble shooting one you can use get_logs method.
deployment.get_logs()

---